# Laboratorio de Criptoanálisis del Cifrado de Vigenère

**Asignatura:**  Seguridad y Protección de Sistemas Informaticos

**Autor:**  Oscar Fernández Rodriguez, Daniel Fernández Calvo, Diego Adriel Segura Ramírez, Juan Carlos Salas Ariza

**Fecha:**  15/12/2025

## Objetivo
Implementar un laboratorio en Python que permita cifrar, descifrar y
romper criptogramas cifrados mediante el criptosistema de Vigenère.

## Fundamento matemático del cifrado de Vigenère

El cifrado de Vigenère puede modelarse algebraicamente utilizando el
grupo aditivo $\mathbb{Z}_{26}$.

Se define el alfabeto como: 
$$
A={A,B,…,Z},∣A∣=26
$$.

Sea $p_i$ la i-ésima letra del texto plano y $k_j$ la j-ésima letra de 
clave, ambas representadas como elementos de $\mathbb{Z}_{26}$. El
cifrado se define como:

$$
c_i = (p_i + k_{i \bmod m}) \bmod 26
$$

y el descifrado como:

$$
p_i = (c_i - k_{i \bmod m}) \bmod 26
$$

donde $m$ es la longitud de la clave.

Este esquema genera un cifrado polialfabético periódico, base del
criptoanálisis clásico del sistema.

## Alfabeto
Para este caso vamos a utilizar el alfabeto en español, que dispone de 26 carácteres, a parte del alfabeto declaramos en este archivo los carácteres especiales para sanitizar los textos, la longitud del alfabeto, y los diccionarios(Uno de número a carácteres y otro de carácteres a números). <br> También se declaran las frecuencias del alfabeto en español.

In [5]:
#alfabeto 
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

#Caracteres especiales
alphSpecials = {
    'Á': 'A',
    'É' : 'E',
    'Í' : 'I',
    'Ó' : 'O',
    'Ú' : 'U',
    'Ä' : 'A',
    'Ë' : 'E',
    'Ï' : 'I',
    'Ö' : 'O',
    'Ü' : 'U',
    'Ñ' : 'GN'
}

#Tamaño del alfabeto
n = len(alphabet)

#Diccionario para  letras : índices 
chNum = {letra: i for i, letra in enumerate(alphabet)}

#Diccionario para índices : letras
numCh = {i: letra for i, letra in enumerate(alphabet)}

#Frecuencias del alfabeto español
alphFreq = {
    'A' : 12.53,
    'B' : 1.42,
    'C' : 4.68,
    'D' : 5.86,
    'E' : 13.68,
    'F' : 0.69,
    'G' : 1.01,
    'H' : 0.70,
    'I' : 6.25,
    'J' : 0.44,
    'K' : 0.02,
    'L' : 4.97,
    'M' : 3.15,
    'N' : 6.71,
    'O' : 8.68,
    'P' : 2.51,
    'Q' : 0.88,
    'R' : 6.87,
    'S' : 7.98,
    'T' : 4.63,
    'U' : 3.93,
    'V' : 0.90,
    'W' : 0.01,
    'X' : 0.22,
    'Y' : 0.90,
    'Z' : 0.52
}

## Vigènere Clases
En el desarrollo del cifrado de Vigenère se han implementado tres clases que permiten abordar el problema desde distintos ángulos: la preparación del texto, el proceso de cifrado y descifrado, y finalmente el criptoanálisis para recuperar la clave, siguiendo los conceptos matemáticos dados en los apuntes de la materia y el ejemplo del cifrado afín.
### Clase Message
La clase Message constituye la base del sistema, ya que se encarga de normalizar el texto de entrada. Esto implica eliminar espacios, convertir todo a mayúsculas y reemplazar caracteres especiales, asegurando que el mensaje y la clave estén en un formato uniforme y compatible con el alfabeto definido. De esta manera, cualquier operación posterior se realiza sobre un contenido limpio y estandarizado, lo que evita inconsistencias en el procesamiento.

In [6]:
from itertools import chain, groupby  # chain lo usamos para concatenar listas y groupby para agrupar

class Message:
    def _flatten(self,listOfLists):   #convierte una lista de listas en una sola lista
        return list(chain.from_iterable(listOfLists))
    
    def _rBlanks(self, strng):   #Elimina los espacios en blanco de un string y lo convierte a mayusculas
        return ''.join(strng.split()).upper()
    
    def _normalize(self, strng):   #normaliza el string: elimina espacios, convierte a mayusculas y reemplaza caracteres especiales
        s = self._rBlanks(strng)  #convertimos a mayusculas y eliminamos espacios
        accum = []
        for ch in s:
            if ch in alphSpecials:
                accum.append(alphSpecials[ch]) #reemplazamos caracteres especiales
            else:
                accum.append(ch)  #dejamos el caracter tal cual

        return list(filter(lambda x: x in alphabet, self._flatten(accum)))  #filtramos solo los caracteres que estan en el alfabeto
    
    def __init__(self, strng):  #constructor de la clase Message
        x=self._normalize(strng)  #normalizamos el string
        self.content = ''.join(x)  #unimos la lista de caracteres en un string
        self.length = len(self.content)  #longitud del mensaje

    def __str__(self):  # mensaje como string
        return self.content


### Clase Encipher
Sobre esta estructura se construye la clase Encipher, que hereda de Message y añade la lógica matemática del cifrado polialfabético. Al recibir un texto y una clave, la clase convierte la clave en índices numéricos y aplica la operación modular característica del Vigenère. <br> El método principal permite tanto cifrar como descifrar, dependiendo del modo seleccionado, y utiliza la clave de manera cíclica para recorrer todo el mensaje. El resultado es un texto transformado que refleja la suma o resta de los índices de las letras del mensaje y la clave, garantizando la reversibilidad del proceso.

In [7]:
class Encipher(Message):
    
    def __init__(self, text, key): #constructor de la clase Encipher recibe el texto y la clave
        super().__init__(text)  #llamamos al contructor de Message para normalizar el texto
        self.key = ''.join(self._normalize(key))  #normalizamos la clave usando el metodo heredado de Message y lo unios en un solo string por si tuviera varias palabras la clave
        self.key_index=[chNum[k] for k in self.key] #convertimos la clave a indices usando el diccionario chNum

    def cipher(self, mode=True): #Método para cifrar o descifrar el mensaje, mode=True para cifrar, False para descifrar

        textcod = [] #almacenamos el texto cifrado o descifrado
        key_length = len(self.key_index)  #longitud de la clave

        for i, char in enumerate(self.content): #vamos pasando por cada caracter del mensaje
            index = chNum[char] #obtenemos el indice del caracter actual
            key_value =self.key_index[i % key_length] # obtenemos el valor de la clave correspondiente al caracter actual
                                                     # lo hacemos ciclico para que si el mensaje es mas largo que la clave, volvamos a empezar desde el principio de la clave

            if mode: #Comprobamos si estamos cifrando o descifrando
                new_index=(index + key_value) % n #ciframos el caracter sumando el indice del caracter y el valor de la clave
            else:
                new_index=(index - key_value) % n #desciframos el caracter restando el valor de la clave al indice del caracter

            textcod.append(numCh[new_index]) #añadimos el nuevo indice a la lista
        
        return ''.join(textcod)  #unimos la lista en un string y lo devolvemos
    

### Clase BreakVigenere
BreakVigenere introduce las técnicas clásicas de criptoanálisis. A partir de un texto cifrado, calcula el índice de coincidencia para estimar la longitud probable de la clave, analiza las frecuencias relativas de los caracteres y aplica pruebas estadísticas como el chi cuadrado para comparar la distribución obtenida con la esperada en el idioma. <br> Con estos métodos, la clase es capaz de determinar los desplazamientos más probables para cada segmento del texto y reconstruir la clave completa. El proceso culmina con la recuperación de la clave y la posibilidad de descifrar el mensaje sin conocerla previamente.

<br>

A continuación que fundamentos matemáticos hemos seguido para quebrantar el criptosistema de Vigènere empezando por explicar su gran debilidad.

### Debilidad de Vigènere
En un principio, se pensaba que Vigènere era inquebrantable, pero eso no es cierto, y se debe a la estructura repetitiva de su clave. <br>

Si un criptoanalista adivina correctamente la longitud de la clave, el texto cifrado puede tratarse como cifras entrelazadas de César. <br>

Una vez conocida la longitud, el texto cifrado se puede reescribir en esa cantidad de columnas, donde cada columna corresponde a una sola letra de la clave. <br>

Como estrategia para romperlo, una vez que tengamos la longitud de la clave se puede dividir el texto en esa cantidad de columnas, donde cada columna corresponda a una sola letra de la clave. <br>

### Determinar la longitud de la Clave
Para encontrar la longitud de la clave, se utiliza la Prueba de Friedman, la cual utiliza el indice de coincidencia

#### Índice de Coincidencia 
El índice de coincidencia es una herramienta estadística que mide la probabilidad de que dos letras tomadas al azar dentro de un texto sean iguales. Su valor depende de la distribución de frecuencias de las letras: en un texto completamente aleatorio, donde todas las letras aparecen con la misma probabilidad, el índice tiende a valores bajos; en cambio, en un texto natural, como el español o el inglés, el índice es más alto porque ciertas letras aparecen con mayor frecuencia que otras. Esta diferencia entre un texto aleatorio y uno con estructura lingüística es lo que permite utilizar el índice de coincidencia en criptoanálisis.
En el caso del cifrado de Vigenère, el índice de coincidencia se emplea para estimar la longitud de la clave. El procedimiento consiste en dividir el texto cifrado en varios segmentos, cada uno correspondiente a una posición de la clave. Si la longitud de la clave es correcta, cada segmento se comporta como un cifrado César independiente y conserva una distribución de letras similar a la del idioma original, lo que se refleja en un índice de coincidencia cercano al esperado para ese idioma. Si la longitud es incorrecta, los segmentos mezclan letras cifradas con distintos desplazamientos y el índice se aproxima al valor de un texto aleatorio. <br>
De esta manera, el índice de coincidencia se convierte en un criterio para seleccionar la longitud más probable de la clave. Una vez determinada, se pueden aplicar otros métodos, como el análisis de frecuencias o la prueba de chi cuadrado, para identificar los desplazamientos específicos de cada posición y reconstruir la clave completa
<br> <br>
**Definición formal:** <br>
$$
IC(s)= \frac{\sum_{i=A}^{c} n_i (n_i - 1)}{N (N - 1)}
$$
Donde: <br>

$n_i$: Frecuencia (conteo) de la letra i en el segmento. <br>
$N$: Longitud total del segmento de texto. <br>
$C$: Tamaño del alfabeto.

**Lógica de ataque(Efoque Matricial)** <br>
Un mejor enfoque para los cifrados de clave repetida es copiar el texto cifrado en las filas de una matriz con tantas columnas como la supuesta longitud de clave y calcular el índice de coincidencia de cada columna. <br>
**Implementación en el código** <br>
La clase BreakVigenere implementa examente este enfoque de lógica de ataque(enfoque matricial)
1. **Método _index_of_coincidence**
```python
#Formula: Suma(n * (n-1)) / (N * (N-1)) 
return sum(v*(v-1) for v in count.values()) / (N*(N-1))
```
2. **Método estimated_key_length**
```python
segments = [self.content[i::L] for i in range(L)]
# [cite_start]Calcula el promedio del IC de cada columna [cite: 526]
con_index = sum(self._index_of_coincidence(seg) for seg in segments)/L
```
### Romper la Clave
Una vez que conocemos la longitud de la clave, el texto cifrado se reescribe en columnas.
Cada columna está compuesta por un texto llano que ha sido cifrado por un único cifrádo César. <br>
Para descubrir la letra de cada columna (el desplazamiento César), se utiliza un análisis de frecuencias comparativo como Chi-Cuadrado
#### Chi-Cuadrado (x²)
El estadístico Chi-Cuadrado $x^2$ es una prueba de hipótesis estadística que se utiliza para determinar si existe una diferencia significativa entre las frecuencias observadas (las que se miden en un texto) y las frecuencias esperadas (las que deberían ocurrir según un modelo teórico) <br>
**Fundamento:** <br>
La prueba $x²$ es una prueba de bondad de ajuste. Compara la distribución de frecuencias de un texto observado (el segmento descifrado con un desplazamiento de prueba) con una distribución de frecuencias esperada. <br><br>
**Definición:** <br>
$$
R_{\chi^2}(M) = \sum_{s \in a} \frac{(O_M(s) - E_A(s))^2}{E_A(s)}
$$
Donde: <br>

$O_M$: Frecuencia observada (relativa o absoluta) de la letra i en el segmento descifrado. <br>

$E_A$: Frecuencia esperada (relativa o absoluta) de la letra i en español (tomada de tu diccionario alphFreq).
<br><br>
**Lógica de ataque:**
 Se prueban los 26 desplazamientos posibles para cada columna. Utilizando métodos similares a los utilizados para descifrar el cifrado César, se pueden descubrir las letras del texto cifrado. <br><br>
**Implementación en el código** <br>
1. **Método chiSquared**
```python
# (Observado - Esperado)[cite_start]^2 / Esperado [cite: 335]
term = (inventory[ch] - expected_prob)**2 / expected_prob
```
2. **Método _best_shift:**
 Itera sobre los 26 posibles desplazamientos, aplica la fórmula $x^2$ y selecciona el mínimo (best_chi), aplicando el principio de ajustar las frecuencias a las del idioma original
3. **Método recover_key:**
 Reconstruye la clave completa uniendo las letras encontradas para cada columna independiente.

In [8]:
class BreakVigenere(Message): #Clase encargada de descubrir la clave 

    def __init__(self, text): #constructor de la clase
        super().__init__(text) #llamamos al constructor de la clase Message

    def _index_of_coincidence(self, seg): #Método para obtener el IC (índice de coincidencia) que probabilidad que dos letras al azar sean iguales
        N = len(seg) #Longitud del texto
        count = {char : seg.count(char) for char in alphabet} #Contamos cuantas veces aparece cada letra en el segmento
        return sum(v*(v-1) for v in count.values()) / (N*(N-1)) if N > 1 else 0 #Cálculo del IC

    def estimated_key_length(self, max_length = 20): #Método para estimar la longitud de la clave
        candidates = [] #Lista de candidatos
        for L in range(1, max_length): #Iterar sobre las posibles longitudes de clave
            segments = [self.content[i::L] for i in range(L)] #Toma el segmento desde i y va de L en L
            con_index = sum(self._index_of_coincidence(seg) for seg in segments)/L #Para cada segmento calcula su IC
            candidates.append((L, con_index)) #Lo añadimos a la lista de candidatos
        candidates.sort(key = lambda x: x[1], reverse = True) #Los ordenamos según el IC promedio más alto
        return candidates[0][0] #Seleccionamos la longitud de clave más probable

    def rfrec(self, strng): #método para calcular la frecuencia relativa de cada caracter en un string
        return {k:len(list(g))/len(strng) for k, g in groupby(''.join(sorted(strng)))} #calculamos la frecuencia relativa de cada caracter

    def chiSquared(self, strng): #método para calcular el chi cuadrado de un string
        inventory = dict.fromkeys(alphabet,0) #inicializamos el inventario de frecuencias
        inventory.update(self.rfrec(strng)) #actualizamos el inventario con las frecuencias del texto
        chDegree = [(len(strng)*(inventory[ch]-alphFreq[ch]))**2/alphFreq[ch] for ch in inventory] #calculamos el chi cuadrado
        return sum(chDegree)  #devolvemos la suma del chi cuadrado

    def _best_shift(self, segment): #método para encontrar el mejor desplazamiento para un segmento
        best_shift = 0 #almacenamos el mejor desplazamiento
        best_chi = float('inf') #inicializamos el mejor chi al infinito
        for shift in range(n): #probamos todos los desplazamientos posibles
            decrypted = ''.join(numCh[(chNum[c]-shift) % n] for c in segment) #desciframos el segmento con el desplazamiento actual
            chi = self.chiSquared(decrypted) #calculamos el chi cuadrado del segmento descifrado
            if chi < best_chi: #si el chi cuadrado es mejor que el mejor encontrado hasta ahora
                best_chi = chi #actualizamos el mejor chi
                best_shift = shift #actualizamos el mejor desplazamiento
        return best_shift #devolvemos el mejor desplazamiento encontrado

    def recover_key(self, key_length): # método para recuperara la clave
        key = [] # Clave
        for i in range(key_length): # Recorremos los segmentos con tamañaos i hasta llegar al tamaño de la clave
            segment = self.content[i::key_length] #obtenemos el segmento
            shift = self._best_shift(segment) #Calculamos el mejor desplazamiento
            key.append(numCh[shift]) #añadimos el valor del caracter
        return ''.join(key) #devolvemos la clave sin espacios

    def break_cipher(self): #método que devuelve la clave
        key_length = self.estimated_key_length() # Obtenemos la longitud de la clave
        key = self.recover_key(key_length) # Obtenemos la clave 
        return key #devolvemos la clave encontrada

## Programa Principal 

In [9]:
import sys

def readTxt(fichero): # Define una función llamada readTxt que recibe como argumento fichero que es una cadena con el nombre o ruta del archivo de texto.
    with open(fichero,'r') as f: #Abre el archivo en modo lectura ('r') y con with asegura que el archivo se cierre automáticamente al terminar el bloque
                                # y f es el objeto archivo.
        lines = f.readlines() # Lee todas las líneas del archivo y el resultado es una lista de strings, donde cada elemento es una línea
    accum = [k for k in lines] # Crea una nueva lista copiando cada línea
    return ''.join(accum) # Une todas las líneas en un solo string.

texto = readTxt("prueba.txt")  #leemos el fichero de entrada
# --- Descifrado automático ---
breaker = BreakVigenere(texto) # Se crea un objeto BreakVigenere texto se guarda internamente como self.content
key_length = breaker.estimated_key_length() # Estimación de la longitud de la clave mediante el Índice de Coincidencia
key = breaker.break_cipher() # Ejecuta el proceso completo de ataque, donde estima la longitud de la clave, divide el texto en segmentos, 
                            # aplica análisis χ² y reconstruye la clave

cipher = Encipher(texto, key) # Crea un objeto Encipher que usa el texto cifrado y la clave recuperada
plaintext = cipher.cipher(False) # Llama al método cipher con False que indica modo descifrado

print("Longitud estimada de la clave:", key_length) # Imprime la longitud estimada de la clave
print("\nClave encontrada:", key) # Imprime la clave recuperada automáticamente
print("\nTexto descifrado:\n") 
print(plaintext) # Imprime el texto descifrado
 


Longitud estimada de la clave: 7

Clave encontrada: CAPITAN

Texto descifrado:

SENORDIJOELCAPITANNEMOMOSTRANDOMELOSINSTRUMENTOSCOLGADOSDELASPAREDESDESUCAMAROTEHEAQUILOSAPARATOSEXIGIDOSPORLANAVEGACIONDELNAUTILUSALIGUALQUEENELSALONLOSTENGOAQUIBAJOMISOJOSINDICANDOMEMISITUACIONYMIDIRECCIONEXACTASENMEDIODELOCEANOALGUNOSDEELLOSLESONCONOCIDOSCOMOELTERMOMETROQUEMARCALATEMPERATURAINTERIORDELNAUTILUSELBAROMETROQUEPESAELAIREYPREDICELOSCAMBIOSDETIEMPOELHIGROMETROQUEREGISTRAELGRADODESEQUEDADDELAATMOSFERAELSTORMGLASSCUYAMEZCLAALDESCOMPONERSEANUNCIALAINMINENCIADELASTEMPESTADESLABRUJULAQUEDIRIGEMIRUTAELSEXTANTEQUEPORLAALTURADELSOLMEINDICAMILATITUDLOSCRONOMETROSQUEMEPERMITENCALCULARMILONGITUDYPORULTIMOMISANTEOJOSDEDIAYDENOCHEQUEMESIRVENPARAESCRUTARTODOSLOSPUNTOSDELHORIZONTECUANDOELNAUTILUSEMERGEALASUPERFICIEDELASAGUASSONLOSINSTRUMENTOSHABITUALESDELNAVEGANTEYSUUSOMEESCONOCIDOREPUSEPEROHAYOTROSAQUIQUERESPONDENSINDUDAALASPARTICULARESEXIGENCIASDELNAUTILUSESECUADRANTEQUEVEORECORRIDOPORUNAAGUJAINMOVILNOESUN

## Texto normalizado y con formato
—Señor —dijo el capitán Nemo mostrándome los instrumentos colgados de las paredes de su camarote—, he aquí los aparatos exigidos por la navegación del Nautilus. Al igual que en el salón, los tengo aquí bajo mis ojos, indicándome mi situación y mi dirección exactas en medio del océano.

Algunos de ellos le son conocidos, como el termómetro, que marca la temperatura interior del Nautilus; el barómetro, que pesa el aire y predice los cambios de tiempo; el higrómetro, que registra el grado de sequedad de la atmósfera; el *storm-glass*, cuya mezcla, al descomponerse, anuncia la inminencia de las tempestades; la brújula, que dirige mi ruta; el sextante, que por la altura del sol me indica mi latitud; los cronómetros, que me permiten calcular mi longitud; y, por último, mis anteojos de día y de noche, que me sirven para escrutar todos los puntos del horizonte cuando el Nautilus emerge a la superficie de las aguas.

Son los instrumentos habituales del navegante, y su uso me es conocido —repuse—. Pero hay otros aquí que responden sin duda a las particulares exigencias del Nautilus. Ese cuadrante que veo recorrido por una aguja inmóvil, ¿no es un manómetro?

—Es un manómetro, en efecto —respondió Nemo—, puesto en comunicación con el agua, cuya presión exterior indica también la profundidad a la que se mantiene mi aparato.

## ¿Es seguro a día de hoy Vigenère?
El cifrado de Vigenère fue considerado irrompible durante siglos, pero su seguridad colapsó gracias al análisis de periodicidad, especialmente con el método del Índice de Coincidencia (IC). Hoy en día, se considera completamente inseguro y su ruptura se realiza en segundos. <br><br>
La razón fundamental de su debilidad radica en dos conceptos clave que violan los principios de la criptografía moderna: periodicidad y reutilización de clave.